<h1><a>Lec 02 NLP基础与扫盲</a></h1>

本节以NLTK为基础配合讲解自然语言处理的原理。

## 目录：
- <a href='#nltk'>NLTK</a>
- <a href='#textprocessing'>文本处理流程</a>
    - <a href='#tokenize'>分词</a>
    - <a href='#standard'>归一化</a>
    - <a href='#stopwords'>停止词</a>
- <a href='#3examples'>NLP经典三案例</a>
    - <a href='#sentiment'>情感分析</a>
    - <a href='#similarity'>文本相似度</a>
    - <a href='#classification'>文本分类</a>
- <a href='#deeplearning'>深度学习加持</a>
    - <a herf='#autoencoder'>Autoencoder</a>
    - <a herf='#word2vec'>Word2Vec</a>

----

<h2><a name='nltk'>1.NLTK</a></h2>

官网：http://www.nltk.org/

Python上著名的自然语言处理库。自带语料库、词性分类库、自带分类、分词，等等功能强大的社区支持，还有N多的简单版wrapper。

textblob就是NLTK的简版版wrapper。
参考网址：https://textblob.readthedocs.io/en/dev/

In [ ]:
# 安装语料库
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


### NLTK功能一览表
|NLTK Modules|Functionality||
|:-|:-:|:-:|
|nltk.corpus|Corpus|语料|
|nltk.tokenize, nltk.stem| Tokenizers, stemmers|分词、词干提取|
|nltk.collocations|t-test, chi-squared, mutual-info|配置|
|nltk.tag|n-gram, backoff, Brill, HMM, TnT||
|nltk.classify, nltk.cluster| Decision tree, Naive Bayes, K-means|分类器|
|nltk.chunk| Regex, n-gram, named entity||
|nltk.parsing| Parsing|treebank 主谓宾，文法树，主要用于NER|
|nltk.sem, nltk.interence| Semantic interpretation|情感分析|
|nltk.metrics| Evaluation metrics|测量|
|nltk.probability| Probability & Estimation|概率|
|nltk.app, nltk.chat| Applications|应用|

### NLTK自带语料库

In [1]:
from nltk.corpus import brown
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [2]:
len(brown.sents())

57340

In [3]:
len(brown.words())

1161192

<h2><a name='textprocessing'>2.文本处理流程</a><h2>

特征工程：将文本分词，（词性标注、词干提取（词性还原）），中止词去除，得到word list，最终数字化。

<img src='./images/nltk01.png' width='50%'/>

<h3>NLTK文本处理流程：一条typical的文本预处理流水线</h3>

<img src='./images/nltk10.png' width='35%'/>

<h3><a name='tokenize'>2.1 分词</a></h3>

Tokenize

将长句子拆成有“意义”的小部件。

In [4]:
import nltk
sentence = 'hello, world!'
tokens = nltk.word_tokenize(sentence)
tokens

['hello', ',', 'world', '!']

#### 中英文NLP的区别
英文天然带有分隔符，用空格“ ”作为分隔符。
中文等非拉丁语言，分词就会有两种：
- rule: 启发式（查字典）
- generative: 机器学习/统计方法：HMM、CRF

<img src='./images/nltk02.png' width='70%'/>

In [5]:
# 中文分词
import jieba
seg_list = jieba.cut('我来到北京清华大学', cut_all=True) # 全模式
print('Full Mode:', '/'.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/来到/北京/清华/清华大学/华大/大学


In [6]:
seg_list = jieba.cut('我来到北京清华大学', cut_all=False) # 精确模式
print('Default Mode:', '/'.join(seg_list))

Default Mode: 我/来到/北京/清华大学


In [7]:
seg_list = jieba.cut('他来到了网易杭研大厦') # 默认是精确模式
print('Default Mode:', ','.join(seg_list))

Default Mode: 他,来到,了,网易,杭研,大厦


【新词识别】：他, 来到, 了了, ⽹网易易, 杭研, ⼤大厦    
(此处，“杭研”并没有在词典中，但是也被Viterbi算法识别出来了了)

In [8]:
seg_list = jieba.cut_for_search('小明硕士毕业于中国科学院计算所，后在日本京都大学深造') # 搜索引擎模式
print(','.join(seg_list))

小明,硕士,毕业,于,中国,科学,学院,科学院,中国科学院,计算,计算所,，,后,在,日本,京都,大学,日本京都大学,深造


#### 分词之后的效果：list of words
<img src='./images/nltk04.png' width='50%'/>

#### 有时候tokenize没那么简单
比如：  
社交网络上，这些乱七八糟的不和语法不合逻辑的语言很多：   
拯救@某人，表情符号，URL，#话题符号

<img src='./images/nltk05.png' width='50%'/>

In [10]:
# 社交网络语言的tokenize
from nltk.tokenize import word_tokenize

tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(word_tokenize(tweet))

['RT', '@', 'angelababy', ':', 'love', 'you', 'baby', '!', ':', 'D', 'http', ':', '//ah.love', '#', '168cm']


#### 对社交网络语言的tokenize，在预处理过程中，用re正则表达式预处理。

#### 正则表达式
对照表：
http://www.regexlab.com/zh/regref.htm

In [11]:
import re
emoticons_str = r'''
(?:
    [:=;] #眼睛
    [oO\-]? # ⿐子
    [D\)\]\(\]/\\OpP] # 嘴
)
'''
regex_str = [emoticons_str, 
            r'<[^>]+>', # HTML tags
            r'(?:@[\w_]+)', # @某⼈
            r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # 话题标签
            r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',
            # URLs
            r'(?:(?:\d+,?)+(?:\.?\d+)?)', # 数字
            r"(?:[a-z][a-z'\-_]+[a-z])", # 含有 - 和 ‘ 的单词
            r'(?:[\w_]+)', # 其他
            r'(?:\S)' # 其他
            ]

In [12]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [13]:
def preprocess(s, lowercase=False):
    def tokenize(s):
        return tokens_re.findall(s)
    tokens = tokenize(s)
    if lowercase:
        tokenize = [token if emotion_re.search(token) else token.lower() for token in tokens]
    return tokens

In [14]:
tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(preprocess(tweet))

['RT', '@angelababy', ':', 'love', 'you', 'baby', '!', ':D', 'http://ah.love', '#168cm']


<h3><a name='standard'>2.2 归一化</a></h3>

- 纷繁复杂的词形

<img src='./images/nltk06.png' width='50%'/>
<img src='./images/nltk07.png' width='70%'/>

#### 词形归一化
- Stemming 词干提取：一般来说，就是把不影响词性的inflection的小尾巴砍掉
    - walking 砍掉ing => walk
    - walked 砍掉ed => walk
    
- Lemmatization 词形归一：把各种类型的词的变形，都归为一个形式（“查表”的方式）
    - went 归一 => go
    - are 归一 => be

#### NLTK实现Stemming：nltk.stem.porter.PorterStemmer

In [15]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
porter_stemmer.stem('maximum')

'maximum'

In [16]:
porter_stemmer.stem('presumably')

'presum'

In [17]:
porter_stemmer.stem('multiply')

'multipli'

In [18]:
porter_stemmer.stem('provision')

'provis'

In [19]:
porter_stemmer.stem('went')

'went'

In [20]:
porter_stemmer.stem('wenting')

'went'

#### NLTK实现Stemming：nltk.stem.SnowballStemmer

In [21]:
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('english')
snowball_stemmer.stem('maximum')

'maximum'

In [22]:
snowball_stemmer.stem('presumably')

'presum'

#### NLTK实现Stemming：nltk.stem.lancaster.LancasterStemmer

In [23]:
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
lancaster_stemmer.stem('maximum')

'maxim'

In [24]:
lancaster_stemmer.stem('presumably')

'presum'

In [25]:
lancaster_stemmer.stem('presumably')

'presum'

#### NLTK实现Lemma: nltk.stem.WordNetLemmatizer

In [26]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('dogs')

'dog'

In [27]:
wordnet_lemmatizer.lemmatize('churches')

'church'

In [28]:
wordnet_lemmatizer.lemmatize('aardwolves')

'aardwolf'

In [29]:
wordnet_lemmatizer.lemmatize('abaci')

'abacus'

In [30]:
wordnet_lemmatizer.lemmatize('hardrock')

'hardrock'

#### Lemma的小问题
<img src='./images/nltk08.png' width='33%'/>

#### NLTK更好地实现Lemma

In [31]:
# 没有PosTag词性标注，默认是 NN(名词)
wordnet_lemmatizer.lemmatize('are')

'are'

In [32]:
wordnet_lemmatizer.lemmatize('is')

'is'

In [33]:
# 加上Pos Tag（词性标注）
wordnet_lemmatizer.lemmatize('is', pos='v')

'be'

In [34]:
wordnet_lemmatizer.lemmatize('are', pos='v')

'be'

#### 词形标注Part-Of-Speech
<img src='./images/nltk09.png' width='70%'/>

In [61]:
# NLTK的词性标注 POS Tag
import nltk
text = nltk.word_tokenize('what does the fox say') # 先分词
print(text)
print(nltk.pos_tag(text)) # 后POS Tag

['what', 'does', 'the', 'fox', 'say']
[('what', 'WDT'), ('does', 'VBZ'), ('the', 'DT'), ('fox', 'NNS'), ('say', 'VBP')]


<h3><a name='stopwords'>2.3 停止词</a></h3>

一千个he，有一千种指代。    
一千个the，有一千种指事。

对于注重理解文本【意思】的应用场景来说，歧义太多。

全体stopwords列表：http://www.ranks.nl/stopwords

#### NLTK去除stopwords
首先，记得在console里面加载一下词库，或者，nltk.download('stopwords')

In [65]:
from nltk.corpus import stopwords
# 先tokenize分词，得到word_list
word_list = nltk.word_tokenize('I am chinese.')
# 然后filter一把
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
print(filtered_words)

['I', 'chinese', '.']


----

<h2><a name='3examples'>3.NLP经典三案例</a></h2>

什么是自然语言处理？

<img src='./images/nltk11.png' width='40%'/>

文本预处理让我们得到了什么？
<img src='./images/nltk12.png' width='40%'/>

NLTK在NLP上的经典应用：情感分析、文本相似度、文本分类。

<h3><a name='sentiment'>3.1 情感分析</a></h3>

工业界还是很受欢迎，常用于行研报告，舆情分析。
<img src='./images/nltk13.png' width='60%'/>

最简单的sentiment dictionary:
- like 1
- good 2
- bad -2
- terrible -3

类似于关键词打分机制

比如：AFINN-111
http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010

工业界确实是有公司用**中文的情感词库**，通过情感词库的关键词打分，最后得出结论。

#### NLTK完成简单的情感分析

In [36]:
sentiment_dictionary = {}
for line in open('./AFINN/AFINN-111.txt'):
    word, score = line.split('\t')
    sentiment_dictionary[word] = int(score)

In [37]:
sentiment_dictionary

{'abandon': -2,
 'abandoned': -2,
 'abandons': -2,
 'abducted': -2,
 'abduction': -2,
 'abductions': -2,
 'abhor': -3,
 'abhorred': -3,
 'abhorrent': -3,
 'abhors': -3,
 'abilities': 2,
 'ability': 2,
 'aboard': 1,
 'absentee': -1,
 'absentees': -1,
 'absolve': 2,
 'absolved': 2,
 'absolves': 2,
 'absolving': 2,
 'absorbed': 1,
 'abuse': -3,
 'abused': -3,
 'abuses': -3,
 'abusive': -3,
 'accept': 1,
 'accepted': 1,
 'accepting': 1,
 'accepts': 1,
 'accident': -2,
 'accidental': -2,
 'accidentally': -2,
 'accidents': -2,
 'accomplish': 2,
 'accomplished': 2,
 'accomplishes': 2,
 'accusation': -2,
 'accusations': -2,
 'accuse': -2,
 'accused': -2,
 'accuses': -2,
 'accusing': -2,
 'ache': -2,
 'achievable': 1,
 'aching': -2,
 'acquit': 2,
 'acquits': 2,
 'acquitted': 2,
 'acquitting': 2,
 'acrimonious': -3,
 'active': 1,
 'adequate': 1,
 'admire': 3,
 'admired': 3,
 'admires': 3,
 'admiring': 3,
 'admit': -1,
 'admits': -1,
 'admitted': -1,
 'admonish': -2,
 'admonished': -2,
 'adopt': 

In [40]:
words = nltk.word_tokenize('I love you! Beatiful Girl!')
# 把这个打分表记录在一个Dict上以后
# 跑一遍整个句子，把对应的值相加
total_score = sum(sentiment_dictionary.get(word, 0) for word in words)
# 有值就是Dict中的值，没有就是0

# 于是就得到了一个sentiment score
total_score

3

显然，这个方法太Naive。
- 新词怎么半？
- 特殊词汇怎么办？
- 更深层次的玩意儿怎么办？

#### 升级：配上ML的情感分析

In [46]:
from nltk.classify import NaiveBayesClassifier

# 随手造训练集
s1 = 'this is a good book'
s2 = 'this is a awesome book'
s3 = 'this is a bad book'
s4 = 'this is a terrible book'

def preprocess(s):
    # Func：句子处理
    # 这里简单的用了split()，把句子中每个单词分开
    # 显然，还有更多的processiing method可以用
    return {word:True for word in s.lower().split()}
    # return⻓长这样:
    # {'this': True, 'is':True, 'a':True, 'good':True, 'book':True}
    # 其中, 前一个叫fname, 对应每个出现的文本单词;
    # 后一个叫fval, 指的是每个⽂本单词对应的值。
    # 这里我们用最简单的True,来表示,这个词『出现在当前的句句⼦子中』的意义。
    # 当然啦, 我们以后可以升级这个方程, 让它带有更加⽜逼的fval, ⽐如 word2vec

In [47]:
# 把训练集给做成标准形式
training_data = [[preprocess(s1), 'pos'],
                 [preprocess(s2), 'pos'],
                 [preprocess(s3), 'neg'],
                 [preprocess(s4), 'neg'],
                ]

# 喂给model
model = NaiveBayesClassifier.train(training_data)

print(model.classify(preprocess('this is a good book')))

pos


<h3><a name='similarity'>3.2 文本相似度</a></h3>

<img src='./images/nltk14.png' width='60%'/>

#### 用元素频率表示文本特征
<img src='./images/nltk15.png' width='60%'/>

#### 余弦定理
$$similarity = \cos \theta = \frac{A \cdot B}{||A|| ||B||}$$

#### Frequency频率统计

In [48]:
import nltk
from nltk import FreqDist

# 做个词库
corpus = 'this is my sentence this is my life this is the day'

# 文本预处理tokenize
# 可以根据需要做任何的preprocessing: stopwords, lemma, stemming , etc
tokens = nltk.word_tokenize(corpus)
print(tokens)

['this', 'is', 'my', 'sentence', 'this', 'is', 'my', 'life', 'this', 'is', 'the', 'day']


In [49]:
# 借用NLTK的FreqDist统计一下文字出现的评率
fdist = FreqDist(tokens)

# fdist就类似于一个Dict,带上某个单词，可以看到它在整个文章中出现的次数。
print(fdist['is'])

3


In [50]:
fdist

FreqDist({'this': 3, 'is': 3, 'my': 2, 'sentence': 1, 'life': 1, 'the': 1, 'day': 1})

In [52]:
# 把最常用的50个单词拿出来
standard_freq_vector = fdist.most_common(50)
size = len(standard_freq_vector)
print(size)
print(standard_freq_vector)

7
[('this', 3), ('is', 3), ('my', 2), ('sentence', 1), ('life', 1), ('the', 1), ('day', 1)]


In [55]:
# Func: 按照出现频率大小，记录下每一个单词的位置
def position_lookup(v):
    res = {}
    counter = 0
    for word in v:
        res[word[0]] = counter
        counter +=1
    return res

# 把标准的单词位置记录下来，得到一个位置对照表
standard_position_dict = position_lookup(standard_freq_vector)
print(standard_position_dict)

{'this': 0, 'is': 1, 'my': 2, 'sentence': 3, 'life': 4, 'the': 5, 'day': 6}


In [57]:
# 这时，如果我们有一个新句子
sentence = 'this is cool'

# 先新建一个跟我们的标准vector同样大小的向量
freq_vector = [0] * size

# 简单的preprocessing
tokens = nltk.word_tokenize(sentence)

# 对这个新句子里面的每一个单词
for word in tokens:
    try:
        # 如果在我们的词库中出现过，那么就在“标准位置“上+1
        freq_vector[standard_position_dict[word]] +=1
    except KeyError:
        continue
    
print(freq_vector)

[1, 1, 0, 0, 0, 0, 0]


<h3><a name='classification'>3.3 文本分类</a></h3>

<img src='./images/nltk16.png' width='60%'/>

#### TF-IDF
- TF: Term Frequency，衡量一个term在文档中出现得有多频繁。

$$TF(t) = \frac{t出现在文档中的次数}{文档中term的总数}$$

- IDF: Inverse Document Frequency，衡量一个term有多重要。
有写词出现的很多，但是明显不是很有卵用。比如：'is','the','and'之类的。
为了平衡，我们把罕见的词的重要性（weight）搞高，把常见词的重要性搞低。

$$IDF(t) = \log_e \frac{文档总数}{含有t的文档总数}$$

$$TF-IDF = TF*IDF$$

<h2><a name='deeplearning'>4.深度学习加持</a></h2>

<h3><a name='autoencoder'>4.1 Autoencoder</a></h3>

<h3><a name='word2vec'>4.2 Word2Vec</a></h3>